In [1]:
pip install numpy opencv-python scikit-learn joblib Flask



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
pip install flask opencv-python-headless numpy scikit-learn joblib


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
import os
import cv2
import numpy as np
from glob import glob
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [17]:
data = 'OneDrive/Desktop/dataset_full/'
categories = ['building', 'forest', 'glacier', 'mountains', 'sea', 'streets']


In [18]:
# Function to load images and labels
def load_images_and_labels(data, categories):
    images = []
    labels = []
    for category in categories:
        category_path = os.path.join(data, category)
        for img_path in glob(os.path.join(category_path, '*.jpg')):
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is not None:
                img = cv2.resize(img, (128, 128))
                images.append(img)
                labels.append(category)
    return images, labels

In [19]:
# Feature extraction functions
def extract_histogram_features(images):
    hist_features = []
    for img in images:
        hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        hist_features.append(hist)
    return np.array(hist_features)

def extract_edge_features(images):
    edge_features = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 100, 200)
        edges = edges.flatten()
        edge_features.append(edges)
    return np.array(edge_features)


In [20]:
# Load images and labels
images, labels = load_images_and_labels(data, categories)

if not images:
    raise ValueError("No images were loaded. Check the dataset path.")

# Extract features
hist_features = extract_histogram_features(images)
edge_features = extract_edge_features(images)

# Combine features
features = np.hstack([hist_features, edge_features])

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


In [21]:
# Dimensionality reduction using PCA
pca = PCA(n_components=20)  # Reduced number of components to speed up training
features_pca = pca.fit_transform(features)

In [22]:
# Train SVM model with RBF kernel
svm = SVC(kernel='rbf', C=1, probability=True)
svm.fit(features_pca, labels_encoded)

SVC(C=1, probability=True)

In [23]:
# Evaluate the model
y_pred = svm.predict(features_pca)
print("Accuracy:", accuracy_score(labels_encoded, y_pred))
print("Classification Report:\n", classification_report(labels_encoded, y_pred, target_names=label_encoder.classes_))

Accuracy: 0.7330791229742613
Classification Report:
               precision    recall  f1-score   support

    building       0.61      0.40      0.48       500
      forest       0.86      0.96      0.90      2745
     glacier       0.49      0.55      0.52       500
   mountains       0.49      0.50      0.50       500
         sea       0.55      0.45      0.49       500
     streets       0.72      0.54      0.62       500

    accuracy                           0.73      5245
   macro avg       0.62      0.57      0.59      5245
weighted avg       0.72      0.73      0.72      5245



In [24]:
# Save the model, PCA object, and label encoder
joblib.dump(svm, 'svm_model.pkl')
joblib.dump(pca, 'pca_object.pkl')
joblib.dump(label_encoder, 'label_encoder.pkl')

['label_encoder.pkl']

In [28]:
from flask import Flask, request, render_template_string
import cv2
import numpy as np
import joblib

app = Flask(__name__)

In [29]:
# Load the trained model, PCA object, and label encoder
svm = joblib.load('svm_model.pkl')
pca = joblib.load('pca_object.pkl')
label_encoder = joblib.load('label_encoder.pkl')

In [30]:
# Feature extraction functions
def extract_histogram_features_single(img):
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

def extract_edge_features_single(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    edges = edges.flatten()
    return edges

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return 'No file part'
        file = request.files['file']
        if file.filename == '':
            return 'No selected file'
        if file:
            img = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
            img = cv2.resize(img, (128, 128))
            
            # Extract features
            hist_features = extract_histogram_features_single(img)
            edge_features = extract_edge_features_single(img)
            
            # Combine features
            features = np.hstack([hist_features, edge_features])
            
            # Apply PCA
            features_pca = pca.transform([features])
            
            # Predict
            prediction = svm.predict(features_pca)
            predicted_label = label_encoder.inverse_transform(prediction)[0]
            
            return f'The image is classified as: {predicted_label}'
    return '''
    <!doctype html>
    <title>Image Classification</title>
    <h1>Upload an Image for Classification</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/May/2024 15:13:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/May/2024 15:13:35] "GET /favicon.ico HTTP/1.1" 404 -
[2024-05-23 15:14:09,315] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\HP\anaconda3\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\HP\anaconda3\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\HP\anaconda3\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\HP\anaconda3\lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_9372\465012230.py", line 32, in upload_file
    features = process_image(file_path)
  File "C:\